# Backtesting a Strategy

In [1]:
# !pip install import_ipynb --quiet
# !pip install anvil-uplink --quiet
# !pip install yfinance --quiet
# !pip install pandas_ta --quiet
# !pip install ipynb --quiet
# !pip install rpyc --quiet

In [2]:
# # pull files from Github
# !git clone https://github.com/gmshroff/algostrats.git

In [3]:
# %cd algostrats

In [30]:
import pandas as pd
import numpy as np
import import_ipynb
from backtest import Backtest
from feeds import BackFeed,DataFeed
from validation import Validate
from featfuncs import feat_aug,add_addl_features_feed,add_ta_features_feed,add_sym_feature_feed
from decision_tree import DecisionTree
import pickle
import pandas as pd

In [31]:
import warnings
warnings.simplefilter("ignore")

Import Strategy - replace with your own class or modify rulestrats

In [32]:
# # upload rulestrats.ipynb 
# from google.colab import files
# uploaded=files.upload()

In [33]:
# change to True if on colab
colab=False

In [5]:
from rulestrats import MomStrat,GapBet,AdaMomCMOADF, RuleStrat
from add_features import add_features
import pickle
from rlagents import RLStratAgentDyn, COLS
from stable_baselines3 import PPO,A2C,DQN
from stable_baselines3.common.vec_env import StackedObservations
from stable_baselines3.common.monitor import Monitor as Mon
from feed_env import Episode
import aspectlib
import os

importing Jupyter notebook from rulestrats.ipynb
importing Jupyter notebook from rlagents.ipynb
importing Jupyter notebook from aiagentbase.ipynb


In [6]:
momStrat=MomStrat()

In [7]:
adaMomStrat=AdaMomCMOADF()

In [8]:
class DTStrat(RuleStrat):
    def __init__(self,th=.00025, model_path='saved_models/dt_hp_11.pkl'):
        super(DTStrat,self).__init__()
        self.logL=[]
        self.th=th
        self.model_type='rule_based'
        self.data_cols='all'
        with open(model_path, 'rb') as f:
            self.model = pickle.load(f)
        
    def check_entry_batch(self,dfD):
        # print(dfD)
        decisionsD={t:0 for t in dfD}
        stopD={t:0 for t in dfD}
        targetD={t:0 for t in dfD}
        
        for t in dfD.keys():
            data=dfD[t]
            last_row = data.iloc[-1:]
            action = self.model.predict(last_row)[0]
            decisionsD[t]=action
            
        return decisionsD,stopD,targetD
    
    def exit_predicate(self,row,df):
        return False
        # print('here!', row, df)
        # data=df
        # r=np.log(data['Close']/data['Close'].shift(1)).dropna()
        # m=r.rolling(20).mean().values
        # if row['quant']>0 and m[-1]<self.th/3: return True
        # elif row['quant']<0 and m[-1]>-self.th/3: return True
        # else: return False

In [34]:
dtStrat = DTStrat(model_path='saved_models/dt_optimal_synthetic_all_hp11.pkl')

In [35]:
dtStrat.model.visualize()

                                                                                                         Index: 0
                                                                                                         Open-Close_polarity >= 1
                                                                                                         Gini: 0.4999364011532591
                                                                                                         Samples: 1064
                                                                                                         Values: {-1: 538, 1: 526}
                                                                                                         Class: -1

                                   Index: 1                                                                                                                                    Index: 2
                                   Open-High_ma_1-Open-High_ma_50_val >= 1  

In [36]:
# from ipynb.fs.defs.mlstrats import MLStrat,MLConfig,ConfigFields

In [37]:
# mlstrat=MLStrat()

Using BackFeed

In [38]:
data=pd.read_csv('./capvol100.csv')

In [39]:
tickers=list(data.iloc[0:10]['ticker'].values)

In [40]:
def get_dist(bt):
    rewards = [bt.results[t][d]['tot'] for t in bt.results for d in bt.results[t]]
    reward_avg = np.mean(rewards)
    reward_std = np.std(rewards)
    return (f'{reward_avg:.1f}\u00B1{reward_std:.1f}')

In [41]:
# feed=BackFeed(tickers=tickers,nd=5,nw=4,synthetic=True)

In [42]:
ans = []
for pickle_path in ['realdata/augdata_01-Jan-2022_5m.csv',
            'realdata/augdata_03-Feb-2022_5m.csv',
          'realdata/augdata_02-Mar-2022_5m.csv', 
          'realdata/augdata_04-Apr-2022_5m.csv', 
          'realdata/augdata_02-May-2022_5m.csv',  
          'realdata/augdata_04-Jul-2022_5m.csv',
          'realdata/augdata_16-Dec-2022_5m.csv']:
    pickle_name = pickle_path.split("/")[-1].rstrip('.csv')
    pickle_name = f'datafeed_{pickle_name}_True_False.pkl'
    # print(f'Loading from {pickle_name}!')
    feed = pickle.load(open(os.path.join('..', 'algodata', 'realdata', pickle_name), 'rb'))
    add_addl_features_feed(feed,tickers=feed.tickers)
    add_sym_feature_feed(feed,tickers=feed.tickers)
    for ticker in feed.data:
        df = feed.data[ticker]
        feed.data[ticker], _, _ = add_features(df)
    bt=Backtest(feed,tickers=feed.tickers,add_features=True,target=.01,stop=.01,txcost=0.001,
            loc_exit=True,scan=False,topk=5,deploy=True,save_dfs=False)
    # Change target to 0.01 for Decision Trees ONLY
    bt.run_all(tickers=feed.tickers,model=dtStrat,verbose=False)
    curr = get_dist(bt)
    print(curr)
    ans.append(curr)

pd.DataFrame({'0': ans})

-0.3±3.0
-0.4±2.5
-0.5±2.3
-0.9±3.5
-0.4±2.1
-0.3±3.2
-0.6±2.5


,0
0,-0.3±3.0
1,-0.4±2.5
2,-0.5±2.3
3,-0.9±3.5
4,-0.4±2.1
5,-0.3±3.2
6,-0.6±2.5


In [43]:
pd.DataFrame({'0': ans})

,0
0,-0.3±3.0
1,-0.4±2.5
2,-0.5±2.3
3,-0.9±3.5
4,-0.4±2.1
5,-0.3±3.2
6,-0.6±2.5


In [19]:
# feed.data[list(feed.data.keys())[10]]['High-Low']

In [22]:
# feed.getDataN('SYN', '20-Mar-2023')

-0.2±2.6


({'VADILALIND.NS': 7.111999275447978,
  'KSL.NS': -6.316427995245086,
  'DSKULKARNI.NS': -1.2,
  'VOLTAS.NS': -7.3534522057819025,
  'SWARAJENG.NS': -2.675482305949615,
  'MANAKSTEEL.NS': -12.959176618492146,
  'TCIDEVELOP.NS': -28.615297110913225,
  'PNC.NS': 1.1775830101767037,
  'MBLINFRA.NS': 18.139252429529122,
  'OMAXE.NS': -3.329869550038823},
 None)

In [25]:
algorithm = PPO
use_alt_data = False
n_steps=2048
win = 5

agent=RLStratAgentDyn(algorithm,monclass=Mon,soclass=StackedObservations,verbose=1,win=win,
                   metarl=True,myargs=(n_steps,use_alt_data), use_cols=COLS)
agent.use_memory=True #depends on whether RL algorithm uses memory for state computation
agent.debug=False
agent.training=False

modelname = 'hp_11.pth'

if modelname and os.path.exists('./saved_models/'+modelname): 
    agent.load_model(filepath='./saved_models/'+modelname)
    print(f'Loading model from {modelname}.')

@aspectlib.Aspect
def my_decorator(*args, **kwargs):
    state,rew,done,exit_type = yield
    args[0].policy.reward((rew,done,{'exit_type':exit_type}))
    return state,rew,done,exit_type

aspectlib.weave(Episode, my_decorator, methods='env_step')

bt=Backtest(feed,tickers=feed.tickers,add_features=False,target=5,stop=5,txcost=0.001,
            loc_exit=True,scan=False,topk=5,deploy=True,save_dfs=False,
            save_func=None)

agent.data_cols=agent.data_cols+['Date']

bt.run_all(tickers=feed.tickers,model=agent,verbose=False)

Using cpu device
Wrapping the env in a DummyVecEnv.
Loading model from hp_11.pth.


In [26]:
bt.returns, get_dist(bt)

-0.9±2.7


({'VADILALIND.NS': -11.93895978688123,
  'KSL.NS': -4.734313589858758,
  'DSKULKARNI.NS': -1.6000000000000003,
  'VOLTAS.NS': -16.87959061070894,
  'SWARAJENG.NS': -7.372421791956777,
  'MANAKSTEEL.NS': 4.329707222166075,
  'TCIDEVELOP.NS': -16.92482413681297,
  'PNC.NS': -53.62881358489917,
  'MBLINFRA.NS': -38.563755854122455,
  'OMAXE.NS': -9.581809650133982},
 None)

In [38]:
from featfuncs import add_global_indices_feed

In [39]:
add_global_indices_feed(feed)

^NYA: No data found for this date range, symbol may be delisted
^IXIC: No data found for this date range, symbol may be delisted
LSEG.L: No data found for this date range, symbol may be delisted
LSEG.L: No data found for this date range, symbol may be delisted


In [ ]:
feed.dates

array(['14-Mar-2023', '15-Mar-2023'], dtype=object)

In [25]:
gapStrat=GapBet()
gapStrat.gdata=feed.gdata

In [26]:
bt.run_all(tickers=feed.tickers,model=gapStrat,verbose=False)

KeyError: 'Close_^NYA'

In [ ]:
bt.returns

In [ ]:
bt.results

Using DataFeed

In [ ]:
# # upload kaggle.json
# from google.colab import files
# uploaded=files.upload()

In [ ]:
# !mkdir /root/.kaggle
# !mv ./kaggle.json /root/.kaggle/.
# !chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# %mkdir data
# %cd data
# !kaggle datasets download -d gmshroff/marketdatafivemin
# !unzip marketdatafivemin.zip
# %cd ..

In [27]:
DATAFILE='augdata_16-Dec-2022_5m.csv'
if not colab: DATAPATH='../algodata/realdata/'
else: DATAPATH='./data/'
df=pd.read_csv(DATAPATH+DATAFILE)

In [28]:
dataFeed=DataFeed(tickers=list(df.ticker.unique()[0:50]),dfgiven=True,df=df)

In [29]:
bt=Backtest(dataFeed,tickers=dataFeed.tickers,target=.05,stop=.01,txcost=0.001,
            loc_exit=True,scan=True,topk=3,deploy=True)

In [30]:
bt.run_all(tickers=dataFeed.tickers,model=momStrat)

In [31]:
bt.results

{'GEECEE.NS': {'05-Dec-2022': {'tot': -1.3090116279069826,
   'rew': [(0, -0.1, 1),
    (1, -0.09999999999999999, 2),
    (2, -0.1, 3),
    (3, -0.1, 4),
    (4, -0.1, 5),
    (5, -0.09999999999999999, 6),
    (6, -0.10000000000000002, 7),
    (7, -0.1, 8),
    (8, -0.1, 9),
    (9, -0.1, 10),
    (10, -0.1, 11),
    (11, -0.1, 12),
    (15, -0.10000000000000002, 16),
    (18, 0.19098837209301764, 40),
    (40, -0.1, 41),
    (41, -0.1, 42)],
   'acts': [(-1, 0, 0, 0),
    (-1, 0, 0, 1),
    (-1, 0, 0, 2),
    (-1, 0, 0, 3),
    (-1, 0, 0, 4),
    (-1, 0, 0, 5),
    (-1, 0, 0, 6),
    (-1, 0, 0, 7),
    (-1, 0, 0, 8),
    (-1, 0, 0, 9),
    (-1, 0, 0, 10),
    (-1, 0, 0, 11),
    (-1, 0, 0, 15),
    (-1, 0, 0, 18),
    (1, 0, 0, 40),
    (1, 0, 0, 41)]}},
 'VSTIND.NS': {'28-Nov-2022': {'tot': -6.154588074739631,
   'rew': [(0, -0.1, 1),
    (1, -0.09999999999999999, 2),
    (2, -0.1, 3),
    (3, -0.1, 4),
    (4, -0.09999999999999999, 5),
    (5, -0.1, 6),
    (6, -0.1, 7),
    (7, -0.

In [32]:
bt.returns

{'GEECEE.NS': -1.3090116279069826,
 'VSTIND.NS': -6.154588074739631,
 'DCM.NS': -34.383941692907435,
 'ZEEMEDIA.NS': -13.925966514330128,
 'CYIENT.NS': -0.6744230769230795,
 'J&KBANK.NS': -27.635214799712124,
 'JPOLYINVST.NS': -21.567921031094276,
 'ZEELEARN.NS': -5.1999999999999975,
 'MBECL.NS': -31.650409569330492,
 'ALMONDZ.NS': -0.0939089567765039,
 'PCJEWELLER.NS': -11.944536486762985,
 'JINDWORLD.NS': -8.827991960234074,
 'CINELINE.NS': -11.498470477784956,
 'SABTN.NS': 0.0,
 'INDIANCARD.NS': -2.2972359812962693,
 'KPRMILL.NS': -2.5367271385678727,
 'CREATIVEYE.NS': 0.0}

Validating remote strategy functions - IGNORE FOR NOW

In [ ]:
try:
    with open('/Users/a112956/MyCode/algo_fin_root/algo_fin_src/anvilcode.txt','r') as f: 
        CODE=f.read()
except:
    CODE=''

In [ ]:
import anvil.server
anvil.server.connect(CODE)

In [ ]:
# Note - first do a backtest using a local strategy; this will also update the feed with
# required features. Only then do remote validation. Else do the following first:
from featfuncs import add_addl_features_feed,add_sym_feature_feed
add_addl_features_feed(feed,tickers=feed.tickers)
add_sym_feature_feed(feed,tickers=feed.tickers)

In [ ]:
dfD={feed.tickers[i]:feed.getDataN(feed.tickers[i],feed.dates[0]) for i in range(4)}

In [ ]:
rpcname='s0_at_a_dot_edu'

In [ ]:
retval=Validate.remote_check_entry_batch(dfD,rpcname)

In [ ]:
list(momStrat.check_entry_batch(dfD))

In [ ]:
Validate(verbose=False).validate(retval=list(momStrat.check_entry_batch(dfD)),dfD=dfD)

In [ ]:
retval=Validate(feed).validate(retval=retval,dfD=dfD)

In [ ]:
retval1=retval.copy()
retval1[0]={'MARUTI.NS': 'five', 'KOTAKBANK.NS': 5, 'NESTLEIND.NS': 5, 'COALINDIA.NS': 5}

In [ ]:
Validate(feed).validate(retval=retval,dfD=dfD)

Backtesting remote strategy

In [ ]:
try:
    with open('/Users/a112956/MyCode/algo_fin_root/algo_fin_src/anvilcode.txt','r') as f: 
        CODE=f.read()
except:
    CODE=''

In [ ]:
import anvil.server
anvil.server.connect(CODE)

In [ ]:
bt=Backtest(feed,tickers=feed.tickers,add_features=True,target=.05,stop=.01,txcost=0.001,
            loc_exit=True,scan=True,topk=3,remote=True,
            data_cols=momStrat.data_cols,rpcname='s0_at_a_dot_edu')

In [ ]:
bt.run_all(tickers=feed.tickers)

In [ ]:
bt.results

In [ ]:
bt.returns

Experiments